In [1]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings('ignore')
load_dotenv('./../.env')

False

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser
from PIL import Image
import base64
import requests
from io import BytesIO

# 모델 불러오기
ollama_model = 'granite3.2-vision'              # ollama pull granite3.2-vision
# ollama_model = 'llava:7b'
# ollama_model = 'llama3.2-vision:11b'
ollama_url = 'http://localhost:11434'

llm = ChatOllama(model=ollama_model, base_url=ollama_url)

# 이미지 인코딩 (Base64 변환)
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 이미지 다운로드    
# image_url = ""
# response = requests.get(image_url)
# image = Image.open(BytesIO(response.content))

In [3]:
system_prompt = SystemMessagePromptTemplate.from_template("""
You are helpful AI assistant who answer user question based on the provided image.
""")

question_format = """
Answer user question based on the provided image ONLY! If you do not know the answer, just say "I do not know".
### Image:
{image}

### Question:
{question}

### Answer:"""

human_prompt = HumanMessagePromptTemplate.from_template(question_format)

messages = [system_prompt, human_prompt]
template = ChatPromptTemplate(messages)

# 체인 생성
qna_chain = template | llm | StrOutputParser()

# 함수 제작
def ask_llm(image_base64, question):
    return qna_chain.invoke({'image':image_base64, 'question':question})

In [7]:
# 이미지를 Base64 로 변환
image_path = "app_icon_p80.png"
image_base64 = encode_image_to_base64(image_path)

# 이미지를 넣어서 질문하기
response = ask_llm(image_base64, "Summarize the contents of this image.")
print(response)


The image appears to be a large, complex string of characters that does not form any recognizable text or pattern. It seems to be a random assortment of letters and symbols without any discernible meaning or structure.


In [14]:
# 랭체인 없이 해보기
import ollama
import base64

# 이미지를 Base64 로 변환
image_path = "img_korean.png"
image_base64 = encode_image_to_base64(image_path)

# 🔹 Ollama에 멀티모달 프롬프트 전달 (Base64 문자열을 직접 포함)
prompt = f"""
I have provided an image in Base64 format below. Analyze this image and answer my question.

Image (Base64):
{image_base64}

Question:
Analyze this image.
"""

response = ollama.chat(
    # model="granite3.2-vision",
    # model="llava:7b",
    model="llama3.2-vision:11b",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)

# 🔹 결과 출력
print(response["message"]["content"])

I can analyze the image, but I must point out that it appears to be a corrupted or encoded image. The text you provided seems to be a binary representation of an image file, possibly a PNG file.

After decoding the binary data, I found that it's actually a base64-encoded string, which is a common way to represent binary data in text format. However, upon further analysis, I realized that this specific encoded string doesn't correspond to any valid or recognizable image.

If you could provide more context about where you got this image from or what kind of image it should be representing, I might be able to help you better. Alternatively, if you can share the original image file or a link to it, I'll do my best to analyze it for you.
